# Controllable generation via RL to let Elon Musk speak ill of DOGE
> How to control text generation through a sentiment classifier.



In [ ]:
# %pip install pfrl@git+https://github.com/voidful/pfrl.git
# %pip install textrl==0.2.15

In [ ]:
import torch
from datasets import load_from_disk
from vc.encodec_model.nar_bart_model import NARBartForConditionalGeneration
from transformers import (AutoTokenizer, BartForConditionalGeneration)
import logging
import sys
import pfrl
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

# define path
base_path = '/work/b0990106x/TextRL'
agent_input_dir = f'{base_path}/data-encodec'
agent_output_dir = f'{base_path}/output'
env_input_dir = agent_output_dir
env_output_dir = agent_input_dir

ar_checkpoint = "lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans"
nar_checkpoint = "lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans"

device = "cuda" if torch.cuda.is_available() else "cpu"
ar_tokenizer = AutoTokenizer.from_pretrained(ar_checkpoint)
ar_model = BartForConditionalGeneration.from_pretrained(ar_checkpoint)
nar_tokenizer = AutoTokenizer.from_pretrained(nar_checkpoint)
nar_model = NARBartForConditionalGeneration.from_pretrained(nar_checkpoint)
ar_model.to(device)

dataset = load_from_disk(agent_input_dir)
# source = dataset[f"src_encodec_0"][0]
# instruction = dataset["instruction"][0]
# transcription = dataset["transcription"][0]
# instruction_ids = ar_tokenizer(instruction)["input_ids"][1 : -1]
# transcription_ids = ar_tokenizer(transcription)["input_ids"][1 : -1]
# src_encodec_ids = ar_tokenizer.convert_tokens_to_ids(
#     [f"v_tok_{u}" for u in dataset[f"src_encodec_0"][0]])
# src_encodec_str = ar_tokenizer.convert_tokens_to_string(
#     [f"v_tok_{u}" for u in dataset[f"src_encodec_0"][0]])


In [ ]:
# prepare data
all_src_encodec_layers = []
all_src_encodec = []
all_instruction = []
all_instruction_ids = []

# data_len = len(dataset)
data_len = 200 # for testing
layer_len = 8

for i in range(layer_len):
    all_src_encodec_layers.append(dataset[f"src_encodec_{i}"])

for i in range(data_len):
    src_encodec = []
    for j in range(layer_len):        
        src_encodec.append(all_src_encodec_layers[j][i])
    all_src_encodec.append(src_encodec)

for i in range(data_len):
    all_instruction.append(dataset["instruction"][i])
    all_instruction_ids.append(ar_tokenizer(all_instruction[i])["input_ids"][1 : -1])
    

In [ ]:
# check data validity
# all data in all_src_encodec must be the numbers instead of strings
# for i in range(data_len):
#     for j in range(layer_len):
#         assert isinstance(all_src_encodec[i][j], list)
#         for k in range(len(all_src_encodec[i][j])):
#             assert isinstance(all_src_encodec[i][j][k], int)

In [ ]:
# run voice conversion model to get the target speech
# import sys
# sys.path.append('/work/b0990106x/TextRL/vc')
# from vc.trainer_encodec_vc_inference import get_ar_prediction, get_ar_prediction_without_writing_files
# from types import SimpleNamespace

# args_predict = SimpleNamespace(
#     output_path = "/work/b0990106x/TextRL/output/example.wav",
#     seed = 0,
#     device = "cuda"
# )    

# single_src_encodec = all_src_encodec[0]
# single_instruction = all_instruction[0]
# print("single_src_encodec: ", single_src_encodec)
# print("single_instruction: ", single_instruction)

# decode_ar = get_ar_prediction(args_predict, ar_model, nar_model, ar_tokenizer, nar_tokenizer, single_src_encodec, single_instruction)
# decode_ar_ids = ar_tokenizer.convert_tokens_to_ids(
#     [f"v_tok_{u}" for u in decode_ar])
# decode_ar_str = ar_tokenizer.convert_tokens_to_string(
#     [f"v_tok_{u}" for u in decode_ar])
    
# print("decode_ar: ", decode_ar)
# print("decode_ar_ids: ", decode_ar_ids)
# print("decode_ar_str: ", decode_ar_str)

In [ ]:
# # demo how the tokenization works

# # source speech before tokenization
# print('source: ', source)
# print('size of source: ', len(source))
# print('src_encodec_ids: ', src_encodec_ids)
# print('size of src_encodec_ids: ', len(src_encodec_ids))
# print('src_encodec_str: ', src_encodec_str)
# print('size of src_encodec_str: ', len(src_encodec_str))
# # source speech after tokenization
# tokens = ar_tokenizer.convert_ids_to_tokens(src_encodec_ids)
# ids = ar_tokenizer.convert_tokens_to_ids(tokens)
# print('ar_tokenizer.convert_ids_to_tokens(src_encodec_ids): ', tokens)
# print('ar_tokenizer.convert_tokens_to_ids(tokens): ', ids)
# print('ar_tokenizer.convert_tokens_to_ids(tokens): ', ids)
# print('size of ar_tokenizer.convert_ids_to_tokens(src_encodec_ids): ', len(tokens))
# # instruction before tokenization
# print(instruction)
# # instruction after tokenization
# print(ar_tokenizer.convert_ids_to_tokens(instruction_ids))
# # transcription before tokenization
# print(transcription)
# # transcription after tokenization
# print(ar_tokenizer.convert_ids_to_tokens(transcription_ids))



In [ ]:
# # inference all data and replace the src_encodec[0] with the decode_ar
# all_decode_ar = []
# all_decode_ar_str = []
# for i in range(data_len):
#     print(f"Processing {i}...")
#     decode_ar = get_ar_prediction_without_writing_files(args_predict, ar_model, nar_model, ar_tokenizer, nar_tokenizer, all_src_encodec[i], all_instruction[i])
#     all_decode_ar.append(decode_ar)
    
#     decode_ar_str = ar_tokenizer.convert_tokens_to_string(
#         [f"v_tok_{u}" for u in decode_ar])
#     all_decode_ar_str.append(decode_ar_str)

In [ ]:
import sys
sys.path.append('/work/b0990106x/TextRL/vc')

from importlib import reload
import textrl
reload(textrl)

from textrl import TextRLEnv,TextRLActor
reload(sys.modules['vc.trainer_encodec_vc_inference'])

from vc.trainer_encodec_vc_inference import get_ar_prediction, get_ar_prediction_without_writing_files

In [ ]:
from NISQA.nisqa.NISQA_model import nisqaModel

class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
        reward = 0
        if finish or len(predicted_list) >= self.env_max_length:
            # single_src_encodec = input_item['src_encodec']
            # single_instruction = input_item['instruction']
            # decode_ar = get_ar_prediction(args_predict, ar_model, nar_model, ar_tokenizer, nar_tokenizer, single_src_encodec, single_instruction)
            
            args_nisqa = {
                'mode': 'predict_file', 
                'pretrained_model': f'{base_path}/NISQA/weights/nisqa.tar', 
                'deg': f'{base_path}/output/example.wav', 
                'data_dir': None, 
                'output_dir': f'{base_path}/NISQA/result',
                'csv_file': None, 
                'csv_deg': None,  
                'num_workers': 0, 
                'bs': 1,
                'ms_channel': None
            }
            args_nisqa['tr_bs_val'] = args_nisqa['bs']
            args_nisqa['tr_num_workers'] = args_nisqa['num_workers']
            
            nisqa = nisqaModel(args_nisqa)
            prediction = nisqa.predict()
            reward = float(prediction['mos_pred'].iloc[0])
            print("input_item : ",input_item['input'])
            print("predicted_list: ", predicted_list)
            print("reward: ", reward) 
                       
        return reward

**fit one example**

In [ ]:
# observation_list = [{'input':src_encodec_str}]
# put all decode_ar to the observation_list
observation_list = []
for i in range(data_len):
    # observation_list.append({'input':all_decode_ar_str[i], 'src_encodec':all_src_encodec[i], 'instruction':all_instruction[i]})
    observation_list.append({'input': "", 'src_encodec': all_src_encodec[i], 'instruction': all_instruction[i]})


In [ ]:
env = MyRLEnv(ar_model, ar_tokenizer, nar_model, nar_tokenizer, observation_input=observation_list, compare_sample=1)
actor = TextRLActor(env, ar_model, ar_tokenizer)
agent = actor.agent_ppo(update_interval=100, minibatch_size=3, epochs=10)

In [ ]:
# predicted_str = actor.predict(observation_list[0])

# # decode the predicted token
# predicted_ids = ar_tokenizer.convert_tokens_to_ids(predicted_str)
# decoded_text = ar_tokenizer.decode(predicted_ids, skip_special_tokens=True)
# print("predicted ids: ", predicted_ids)
# print("decoded text: ", decoded_text)

In [ ]:
pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=4,
    eval_n_steps=None,
    eval_n_episodes=2,       
    train_max_episode_len=5,  
    eval_interval=10,
    outdir='train_200', 
)

loading the best result and predict.

In [ ]:
agent.load("./elon_musk_dogecoin/best")

In [ ]:
actor.predict(observation_list[0])